# Personalizing biobert

## imports

In [1]:
from ast import literal_eval
from itertools import chain
from sklearn.metrics import precision_recall_fscore_support
from tqdm.notebook import tqdm, trange
from sklearn.model_selection import StratifiedKFold
import torch
from transformers import AutoModel, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm.autonotebook import tqdm

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split



import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import FunctionTransformer#, ColumnTransformer

[nltk_data] Downloading package stopwords to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tara-
[nltk_data]     sophiatumbraegel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [34]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
# load data
import pandas as pd
df = pd.read_csv('../data/raw/mtsamples.csv')
df.transcription=df.transcription.astype(str)
#print(df.columns)
df_test = df.head(15)
df_test.head(2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with complaint of allergies.,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female presents with complaint of allergies. She used to have allergies when she lived in Seattle but she thinks they are worse here. In the past, she has tried Claritin, and Zyrtec. Both worked for short time but then seemed to lose effectiveness. She has used Allegra also. She used that last summer and she began using it again two weeks ago. It does not appear to be working very well. She has used over-the-counter sprays but no prescription nasal sprays. She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals: Weight was 130 pounds and blood pressure 124/78.,HEENT: Her throat was mildly erythematous without exudate. Nasal mucosa was erythematous and swollen. Only clear drainage was seen. TMs were clear.,Neck: Supple without adenopathy.,Lungs: Clear.,ASSESSMENT:, Allergic rhinitis.,PLAN:,1. She will try Zyrtec instead of Allegra again. Another option will be to use loratadine. She does not think she has prescription coverage so that might be cheaper.,2. Samples of Nasonex two sprays in each nostril given for three weeks. A prescription was written as well.","allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic,"
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climbing stairs, difficulty with airline seats, tying shoes, used to public seating, and lifting objects off the floor. He exercises three times a week at home and does cardio. He has difficulty walking two blocks or five flights of stairs. Difficulty with snoring. He has muscle and joint pains including knee pain, back pain, foot and ankle pain, and swelling. He has gastroesophageal reflux disease.,PAST SURGICAL HISTORY:, Includes reconstructive surgery on his right hand 13 years ago. ,SOCIAL HISTORY:, He is currently single. He has about ten drinks a year. He had smoked significantly up until several months ago. He now smokes less than three cigarettes a day.,FAMILY HISTORY:, Heart disease in both grandfathers, grandmother with stroke, and a grandmother with diabetes. Denies obesity and hypertension in other family members.,CURRENT MEDICATIONS:, None.,ALLERGIES:, He is allergic to Penicillin.,MISCELLANEOUS/EATING HISTORY:, He has been going to support groups for seven months with Lynn Holmberg in Greenwich and he is from Eastchester, New York and he feels that we are the appropriate program. He had a poor experience with the Greenwich program. Eating history, he is not an emotional eater. Does not like sweets. He likes big portions and carbohydrates. He likes chicken and not steak. He currently weighs 312 pounds. Ideal body weight would be 170 pounds. He is 142 pounds overweight. If ,he lost 60% of his excess body weight that would be 84 pounds and he should weigh about 228.,REVIEW OF SYSTEMS: ,Negative for head, neck, heart, lungs, GI, GU, orthopedic, and skin. Specifically denies chest pain, heart attack, coronary artery disease, congestive heart failure, arrhythmia, atrial fibrillation, pacemaker, high cholesterol, pulmonary embolism, high blood pressure, CVA, venous insufficiency, thrombophlebitis, asthma, shortness of breath, COPD, emphysema, sleep apnea, diabetes, leg and foot swelling, osteoarthritis, rheumatoid arthritis, hiatal hernia, peptic ulcer disease, gallstones, infected gallbladder, pancreatitis, fatty liver, hepatitis, hemorrhoids, rectal bleeding, polyps, incontinence of stool, urinary stress incontinence, or cancer. Denies cellulitis, pseudotumor cerebri, meningitis, or 

## cleaning

In [50]:
import string
def cleaning(sentence):
    '''This function takes a column and cleans it by removing punctuation, stopwords, and lemmatizing
    NEXT STEPS : personalize the stop words list, check for different aspects of the words
    '''
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase 
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize 
    stop_words = set(stopwords.words('english')) ## define stopwords
    
    tokenized_sentence_cleaned = [ ## remove stopwords
        w for w in tokenized_sentence if not w in stop_words
    ]

    lemmatized = [
        WordNetLemmatizer().lemmatize(word, pos = "v") 
        for word in tokenized_sentence_cleaned
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence
df_test["transcription"] = df_test["transcription"].apply(cleaning)

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/2744472210.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["transcription"] = df_test["transcription"].apply(cleaning)


In [49]:
def convert(lst_kw):
    '''This function converts the keywords to a list of keywords'''
    list_keywords = lst_kw.split(',')
    return list_keywords
df_test['keywords']=df_test['keywords'].fillna("")
df_test['keywords_list']= df_test['keywords'].apply(lambda x: x.split(','))
# NEXT STEP: clean keyowrds - lemmatize, lowercase

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/3971471173.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['keywords']=df_test['keywords'].fillna("")
/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/3971471173.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['keywords_list']= df_test['keywords'].apply(lambda x: x.split(','))


In [53]:
def location_indices(stringstext, check_list):
    '''this function finds the location of the keywords in the string'''
    
    res = dict()
    for ele in check_list :
        if ele in stringstext:
            # getting front index
            strt = stringstext.index(ele)
            
            # getting ending index
            res[ele] = [strt, strt + len(ele) - 1]
    return res.values()
        
df_test['location'] = df_test.apply(lambda x: location_indices(x.transcription, x.keywords_list), axis=1) 

/var/folders/7p/lw9128713_9433llvvhnlv680000gn/T/ipykernel_1971/2437295253.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['location'] = df_test.apply(lambda x: location_indices(x.transcription, x.keywords_list), axis=1)


In [54]:
#Kfold
import numpy as np
frames = []
df_split = np.array_split(df_test, 5)
for split in range(0, 5):
    df_split[split]['kfold'] = split
    frames.append(df_split[split])
dfx = pd.concat(frames)
dfx

#find max len der texte
max_len = df_test['transcription'].map(lambda x: len(x)).max()
max_len
pd.set_option('display.max_colwidth', None)
dfx.head(1)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location,kfold
0,0,A 23-year-old white female presents with complaint of allergies.,Allergy / Immunology,Allergic Rhinitis,subjective yearold white female present complaint allergies use allergies live seattle think worse past try claritin zyrtec work short time seem lose effectiveness use allegra also use last summer begin use two weeks ago appear work well use overthecounter spray prescription nasal spray asthma doest require daily medication think flare upmedications medication currently ortho tricyclen allegraallergies know medicine allergiesobjectivevitals weight pound blood pressure heent throat mildly erythematous without exudate nasal mucosa erythematous swell clear drainage see tms clearneck supple without adenopathylungs clearassessment allergic rhinitisplan try zyrtec instead allegra another option use loratadine think prescription coverage might cheaper sample nasonex two spray nostril give three weeks prescription write well,"allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic,","[allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic, ]","([633, 650], [49, 58], [287, 293], [642, 650], [275, 280], [492, 504], [167, 174], [633, 641], [0, -1])",0


## config 

In [2]:
#configuration

class config:
    MAX_LEN = 42 # 416
    TRAIN_BATCH_SIZE = 8
    VALID_BATCH_SIZE = 8
    EPOCHS = 5
    
    # BERT_PATH = "../input/bert-base-uncased/" 
    # MODEL_PATH = "model.bin"
    # TOKENIZER = tokenizers.BertWordPieceTokenizer(f"{BERT_PATH}/vocab.txt" ,lowercase = True)
    
    BERT_PATH = 'emilyalsentzer/Bio_ClinicalBERT'
    MODEL_PATH = 'emilyalsentzer/Bio_ClinicalBERT'
    TOKENIZER = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    DROPOUT = 0.2
    MAX_GRAD_NORM = 1.0
    LEARNING_RATE = 1e-5

## tokenizer

In [3]:
def loc_list_to_ints(loc_list):
    tuples = []
    for sublist in loc_list:
        tuples.append(tuple(sublist))
    return tuples

In [92]:
def process_data_tokenize(feature_text, annotation, location, tokenizer, max_len):    ##X , Y, selected_text  
    '''this function tokenizes the data'''
    location_list = loc_list_to_ints(location)        
    char_targets = [0] * len(feature_text) #creating empty list(all zeros) of character;it will be made 1 if annotation in text   
    for loc,anno in zip(location_list,annotation): 
        len_st = loc[1] - loc[0]

        idx0 = None
        idx1 = None
        for ind in (i for i, e in enumerate(feature_text) if (e == anno[0] and i == loc[0])):
            if feature_text[ind: ind+len_st] == anno:

                idx0 = ind
                idx1 = ind + len_st - 1
                if idx0 != None and idx1 != None:
                    for ct in range(idx0, idx1 + 1):
                        char_targets[ct] = 1  #replacing zeros with 1 if that part of the text is selected text
        
                break
    print('in process function')
    tokenized_input = tokenizer.encode_plus(feature_text,
                                            add_special_tokens=True, # this ads tokens like [CLS]- end and [SEP]- start 
                                            return_attention_mask=True, 
                                            return_offsets_mapping=True,
                                            padding =  'max_length',
                                            truncation = True,
                                            max_length=42,
                                            return_tensors = 'pt')
    
    #input_ids = tokenized_input['input_ids'].flatten()
    input_ids = tokenized_input.get('input_ids')
    mask = tokenized_input.get('attention_mask')
    token_type_ids = tokenized_input.get('token_type_ids')
    offsets = tokenized_input.get('offset_mapping')
    
    input_ids = torch.cat(input_ids, dim=0)
    mask = torch.cat(mask, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)
    offsets = torch.cat(offsets, dim=0)
    # input ids - id representation of each token - could decode by tokenizer.decode(input_ids[0])
    # tokentype ids - 0 for first sentence, 1 for second sentence
    # attention mask - 1 for tokens that are not [PAD] and 0 for [PAD] tokens
    
    #print('input_ids.specific', input_ids[0])
    print('input_ids.shape', input_ids.shape)
    
    
    target_idx = []
    for value in offsets:
        for j, (offset1, offset2) in enumerate(value):
            if sum(char_targets[offset1: offset2]) > 0:
                target_idx.append(j)

    #creating label
    ignore_idxes = np.where(np.array(token_type_ids) != 1)[0]
    label = np.zeros(offsets.size())#, dtype=np.float32)]))
    label[ignore_idxes] = -1
    label[target_idx] = 1
    print('label', label)
    
    return {
    'ids': input_ids,
    'mask': mask,
    'token_type_ids': token_type_ids,
    'labels': label,
    'offsets': offsets
}

In [93]:
import numpy as np
#output = process_data_tokenize(example["feature_text"],example["annotation"],example["location"],config.TOKENIZER,config.MAX_LEN)

#for key in output.keys():
#    print(key)
#    print(output[key])
#    print("=" * 100)
    
#output


df_test.head(1)
df_traindict = {
    "feature_text": df_test.transcription,
    "location": df_test.location,
    "annotation": df_test.keywords_list
}
#list(df_traindict['feature_text'])
output_train = process_data_tokenize(list(df_traindict["feature_text"]),list(df_traindict["annotation"]),list(df_traindict["location"]),config.TOKENIZER,config.MAX_LEN)
#output_train = process_data_tokenize(df_traindict["feature_text"],df_traindict["annotation"],df_traindict["location"],config.TOKENIZER,config.MAX_LEN)
output_train

TypeError: unsupported operand type(s) for -: 'list' and 'list'

## Data Loader

In [56]:
#Data loader
from torch.utils.data import Dataset, DataLoader
class NBMEDataset(Dataset):
    def __init__(self,  feature_text, annotation, location):   #text(X) #label(Y), #selected_text #start:end
        #self.pn_history = pn_history
        self.feature_text = list(feature_text)
        self.annotation = list(annotation)
        self.location = list(location)
        self.tokenizer = config.TOKENIZER
        self.max_len = config.MAX_LEN
        
    def __len__(self):
        return len(self.feature_text)
        
    def __getitem__(self, item):
        data = process_data_tokenize(
            #self.pn_history[item],
            self.feature_text[item],
            self.annotation[item],
            self.location[item],
            self.tokenizer,
            self.max_len
        )
        return {
            'ids': torch.tensor(data["ids"]), #input_ids
            'mask': torch.tensor(data["mask"], dtype=torch.long), #attention_mask
            'token_type_ids': torch.tensor(data["token_type_ids"], dtype=torch.long), #segment_ids
            'labels': torch.tensor(data["labels"], dtype=torch.long), 
            'offsets': torch.tensor(data["offsets"], dtype=torch.long)
        }
        

## Model

In [29]:
import transformers
class NBMEModel(transformers.BertPreTrainedModel):    #torch.nn.Module
    def __init__(self,conf):
        super(NBMEModel,self).__init__(conf)
        self.bert = transformers.BertModel.from_pretrained(config.BERT_PATH, config=conf)
        self.dropout = torch.nn.Dropout(config.DROPOUT)
        self.classifier = torch.nn.Linear(768, 1)
        torch.nn.init.normal_(self.classifier.weight, std=0.02) 
        
    def forward(self, ids, mask, token_type_ids):
        sequence_out = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)[0] #last_hidden_state
        batch_size, max_len, feat_dim = sequence_out.shape
        sequence_output = self.dropout(sequence_out)
        logits = self.classifier(sequence_output)
        logits = logits.squeeze(-1) 
        return logits

In [11]:
#utility function
class AverageMeter:
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        
#loss function
def loss_fn(logits, labels):
    print('in loss function')
    #labels = labels.unsqueeze(1)
    loss_fct = torch.nn.BCEWithLogitsLoss(reduction = "none")
    loss = loss_fct(logits,labels)
    return loss

## training function

In [90]:
# in a next step we use data loader to load the data - https://blog.paperspace.com/dataloaders-abstractions-pytorch/
# in a next step with the help of the optimizer we will update the weights of the model
#training function


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def train_fn(dataloader, model, optimizer, scheduler=None): #before: dataloader
    model.train()
    losses = AverageMeter() 
    print('in function train_fn')

    for batch_idx, batch in enumerate(tqdm(dataloader, total=len(dataloader))):
        for k, v in batch.items():
            batch[k] = v.to(DEVICE)

        optimizer.zero_grad()
        
        #ids = batch['ids'].squeeze(1)
    
        #output = torch.squeeze(input)
        logits = model(ids=batch['ids'], mask=batch['mask'], token_type_ids=batch['token_type_ids'])
        #model(ids=batch['ids'], mask=batch['mask'], token_type_ids=batch['token_type_ids'])
        loss = loss_fn(logits,batch["labels"])
        loss = loss.mean()
        # loss = torch.masked_select(loss, labels > -1.0).mean()
        # losses.update(loss.item(),ids.size(0))
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        losses.update(loss.item(), batch['ids'].size(0))
        dataloader.set_postfix(loss=losses.avg) #tk. bfefore
        #or like this - https://www.kaggle.com/code/daisybbb/pytorch-pubmedbert-infer


        torch.nn.utils.clip_grad_norm_(model.parameters(), config.MAX_GRAD_NORM)
        optimizer.step()
        scheduler.step() ## Update learning rate schedule
    
    # #output = torch.argmax(torch.softmax(logits, dim=2),dim=2).cpu().detach().numpy()
        tqdm(dataloader, total=len(dataloader)).set_postfix(loss=losses.avg)
        
    return losses.avg


In [34]:
 # train model
 #adam w - https://medium.com/@nishantnikhil/adam-optimizer-notes-ddac4fd7218
 #other optimizer https://medium.com/geekculture/a-2021-guide-to-improving-cnns-optimizers-adam-vs-sgd-495848ac6008
 #https://firiuza.medium.com/optimizers-for-training-neural-networks-e0196662e21e
 #https://medium.com/@ThiyaneshwaranG/types-of-optimizers-in-deep-learning-from-gradient-descent-to-adam-1572e657c8c5
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup

#-------------------
df_test.head(1)
df_traindict = {
    "feature_text": df_test.transcription,
    "location": df_test.location,
    "annotation": df_test.keywords_list
}
list(df_traindict['feature_text'])
output_train = process_data_tokenize(list(df_traindict["feature_text"]),list(df_traindict["annotation"]),list(df_traindict["location"]),config.TOKENIZER,config.MAX_LEN)
#output_train = process_data_tokenize(df_traindict["feature_text"],df_traindict["annotation"],df_traindict["location"],config.TOKENIZER,config.MAX_LEN)
output_train



#-----------------#
model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
model_config.output_hidden_states = True
model = NBMEModel(conf=model_config)
model.to(DEVICE)
    
#num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
param_optimizer = list(model.named_parameters())
#print('param_optimizer', param_optimizer)
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]

optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
optimizer = AdamW(optimizer_parameters, lr=3e-5)
scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=2
    )
#-----------------#
train_loss = train_fn(x, model, optimizer, scheduler=scheduler)
#train_loss_data.append(train_loss)
print(f"Train loss: {train_loss}")

TypeError: 'dict_values' object is not subscriptable

## evaluation function

In [ ]:
#evaluation function
def eval_fn(dataloader, model): #before: dataloader
    model.eval()
    losses = AverageMeter() # Computes and stores the average and current value

    with torch.no_grad():
        tk = tqdm(dataloader, total=len(dataloader)) 
        #tk = tqdm(data, total=5)
        
        # ids = data['ids']
        # token_type_ids = data["token_type_ids"]
        # mask = data["mask"]
        # labels = data["labels"]
        # offsets = data["offsets"]#    
        # labels = labels.unsqueeze(0)

        for batch, data in enumerate(tk):

            ids = data['ids']
            token_type_ids = data["token_type_ids"]
            mask = data["mask"]
            labels = data["labels"]
            offsets = data["offsets"]

            ids = ids.to(DEVICE, dtype=torch.long)
            token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
            mask = mask.to(DEVICE, dtype=torch.long)
            labels = labels.to(DEVICE, dtype=torch.float64)

            logits = model(ids=ids, mask=mask, token_type_ids=token_type_ids) #last_hidden_state
            
            loss = loss_fn(logits, labels)
            loss = torch.masked_select(loss, labels > -1.0).mean()
            losses.update(loss.item(),ids.size(0))
            tk.set_postfix(loss=losses.avg)
        
        return losses.avg

In [ ]:
df_test.head(1)
df_traindict = {
    "feature_text": df_test.transcription,
    "location": df_test.location,
    "annotation": df_test.keywords_list
}
list(df_traindict['feature_text'])
output_train = process_data_tokenize(list(df_traindict["feature_text"]),list(df_traindict["annotation"]),list(df_traindict["location"]),config.TOKENIZER,config.MAX_LEN)
#output_train = process_data_tokenize(df_traindict["feature_text"],df_traindict["annotation"],df_traindict["location"],config.TOKENIZER,config.MAX_LEN)
output_train

TypeError: 'dict_values' object is not subscriptable

## Run training

In [91]:

#training
import time
import numpy as np
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
def run(fold):
    
    train_loss_data, valid_loss_data = [], []
    best_loss = np.inf
    since = time.time()
   
    df_train = dfx[dfx.kfold != fold].reset_index(drop=True) 
    df_valid = dfx[dfx.kfold == fold].reset_index(drop=True)
    
    train_dataset = NBMEDataset(
        feature_text=df_train.transcription.values,
        annotation=df_train.keywords_list.values,
        location=df_train.location.values
        
    )
    
    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=config.TRAIN_BATCH_SIZE,
        num_workers=0
    )

    valid_dataset = NBMEDataset(
        feature_text=df_valid.transcription.values,
        annotation=df_valid.keywords_list.values,
        location=df_valid.location.values
    )

    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=config.VALID_BATCH_SIZE,
        num_workers=0
    )

    model_config = transformers.BertConfig.from_pretrained(config.BERT_PATH)
    model_config.output_hidden_states = True
    model = model = model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
    # NBMEModel(conf=model_config)
    model.to(DEVICE)
    
    num_train_steps = int(len(df_train) / config.TRAIN_BATCH_SIZE * config.EPOCHS)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
    ]
    optimizer = AdamW(optimizer_parameters, lr=config.LEARNING_RATE)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=0, 
        num_training_steps=num_train_steps
    )

    best_loss = np.inf
    
    for i in range(config.EPOCHS):
        print("Epoch: {}/{}".format(i + 1, config.EPOCHS))

        # train model
        train_loss = train_fn(train_data_loader
                              , model, optimizer, scheduler=scheduler)
        train_loss_data.append(train_loss)
        print(f"Train loss: {train_loss}")

        # evaluate model - not now
      #  valid_loss = eval_fn(valid_data_loader, model)
      #  valid_loss_data.append(valid_loss)
      #  print(f"Valid loss: {valid_loss}")


       # if valid_loss < best_loss:
        #    best_loss = valid_loss
        torch.save(model.state_dict(), "model_fold1.bin")


        #time_elapsed = time.time() - since
        #print('Training completed in {:.0f}m {:.0f}s'.format(
        #    time_elapsed // 60, time_elapsed % 60))
    
    
run(fold=0)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of Ada

Epoch: 1/5
in function train_fn


  0%|          | 0/2 [00:00<?, ?it/s]

in process function


TypeError: cat() received an invalid combination of arguments - got (Tensor, dim=int), but expected one of:
 * (tuple of Tensors tensors, int dim, *, Tensor out)
 * (tuple of Tensors tensors, name dim, *, Tensor out)


# Keyword bert without personalization

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer

# doc = """
#          Supervised learning is the machine learning task of 
#          learning a function that maps an input to an output based 
#          on example input-output pairs.[1] It infers a function 
#          from labeled training data consisting of a set of 
#          training examples.[2] In supervised learning, each 
#          example is a pair consisting of an input object 
#          (typically a vector) and a desired output value (also 
#          called the supervisory signal). A supervised learning 
#          algorithm analyzes the training data and produces an 
#          inferred function, which can be used for mapping new 
#          examples. An optimal scenario will allow for the algorithm 
#          to correctly determine the class labels for unseen 
#          instances. This requires the learning algorithm to  
#          generalize from the training data to unseen situations 
#          in a 'reasonable' way (see inductive bias).
#       """



def embed(input):
      n_gram_range = (1, 3)
      stop_words = "english"
      # Extract candidate words/phrases - count vectorizer works with the time a word appears in the document
      count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([input])
      candidates = count.get_feature_names()
      #Next, we convert both the document as well as the candidate keywords/keyphrases to numerical data.
      model = SentenceTransformer('distilbert-base-nli-mean-tokens')
      #this transofrms the document into a vector, and the next transforms the candidate words into vectors
      doc_embedding = model.encode([input])
      candidate_embeddings = model.encode(candidates)
      return doc_embedding, candidate_embeddings, candidates

I use Distilbert as it has shown great performance in similarity tasks, which is what we are aiming for with keyword/keyphrase extraction!

In the final step, we want to find the candidates that are most similar to the document. We assume that the most similar candidates to the document are good keywords/keyphrases for representing the document.

To calculate the similarity between candidates and the document, we will be using the cosine similarity between vectors as it performs quite well in high-dimensionality:

In [68]:


top_n = 15
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['supervised learning example',
 'analyzes training data',
 'machine learning task',
 'examples supervised learning',
 'requires learning algorithm',
 'signal supervised learning',
 'learning algorithm',
 'learning function maps',
 'supervised learning machine',
 'supervised learning algorithm',
 'algorithm analyzes training',
 'learning algorithm generalize',
 'learning machine learning',
 'learning algorithm analyzes',
 'algorithm generalize training']

There is a reason why similar results are returned… they best represent the document! If we were to diversify the keywords/keyphrases then they are less likely to represent the document well as a collective.

Thus, the diversification of our results requires a delicate balance between the accuracy of keywords/keyphrases and the diversity between them.

There are two algorithms that we will be using to diversify our results:

Max Sum Similarity
Maximal Marginal Relevance
Max Sum Similarity
The maximum sum distance between pairs of data is defined as the pairs of data for which the distance between them is maximized. In our case, we want to maximize the candidate similarity to the document whilst minimizing the similarity between candidates.

To do this, we select the top 20 keywords/keyphrases, and from those 20, select the 5 that are the least similar to each other:
https://towardsdatascience.com/keyword-extraction-with-bert-724efca412ea

In [75]:
import numpy as np
import itertools
from sklearn.metrics.pairwise import cosine_similarity


def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, word_embeddings)
    distances_candidates = cosine_similarity(word_embeddings, 
                                            word_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [words[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

Maximal Marginal Relevance
The final method for diversifying our results is Maximal Marginal Relevance (MMR). MMR tries to minimize redundancy and maximize the diversity of results in text summarization tasks. Fortunately, a keyword extraction algorithm called EmbedRank has implemented a version of MMR that allows us to use it for diversifying our keywords/keyphrases.

We start by selecting the keyword/keyphrase that is the most similar to the document. Then, we iteratively select new candidates that are both similar to the document and not similar to the already selected keywords/keyphrases:

In [71]:

import numpy as np

def mmr(doc_embedding, word_embeddings, words, top_n, diversity):

    # Extract similarity within words, and between words and the document
    word_doc_similarity = cosine_similarity(word_embeddings, doc_embedding)
    word_similarity = cosine_similarity(word_embeddings)

    # Initialize candidates and already choose best keyword/keyphras
    keywords_idx = [np.argmax(word_doc_similarity)]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    for _ in range(top_n - 1):
        # Extract similarities within candidates and
        # between candidates and selected keywords/phrases
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # Calculate MMR
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # Update keywords & candidates
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [86]:
#for dataframe
list_output = []
for column in df_test.transcription:
    doc_embedding, candidate_embeddings, candidates = embed(column)
    max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)
    list_output.append(mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2))
df_test['oup'] = list_output    

/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/skle

In [87]:
#for custom input
doc_embedding, candidate_embeddings, candidates = embed(doc)
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,keywords_list,location,oup
0,0,A 23-year-old white female presents with complaint of allergies.,Allergy / Immunology,Allergic Rhinitis,subjective yearold white female present complaint allergies use allergies live seattle think worse past try claritin zyrtec work short time seem lose effectiveness use allegra also use last summer begin use two weeks ago appear work well use overthecounter spray prescription nasal spray asthma doest require daily medication think flare upmedications medication currently ortho tricyclen allegraallergies know medicine allergiesobjectivevitals weight pound blood pressure heent throat mildly erythematous without exudate nasal mucosa erythematous swell clear drainage see tms clearneck supple without adenopathylungs clearassessment allergic rhinitisplan try zyrtec instead allegra another option use loratadine think prescription coverage might cheaper sample nasonex two spray nostril give three weeks prescription write well,"allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic,","[allergy / immunology, allergic rhinitis, allergies, asthma, nasal sprays, rhinitis, nasal, erythematous, allegra, sprays, allergic, ]","([633, 650], [49, 58], [287, 293], [642, 650], [275, 280], [492, 504], [167, 174], [633, 641], [0, -1])","[clearneck supple adenopathylungs, allergies live seattle, medicine allergiesobjectivevitals, adenopathylungs clearassessment allergic, nostril weeks prescription]"
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,past medical history difficulty climb stairs difficulty airline seat tie shoe use public seat lift object floor exercise three time week home cardio difficulty walk two block five flight stairs difficulty snore muscle joint pain include knee pain back pain foot ankle pain swell gastroesophageal reflux diseasepast surgical history include reconstructive surgery right hand years ago social history currently single ten drink year smoke significantly several months ago smoke less three cigarettes dayfamily history heart disease grandfathers grandmother stroke grandmother diabetes deny obesity hypertension family memberscurrent medications noneallergies allergic penicillinmiscellaneouseating history go support group seven months lynn holmberg greenwich eastchester new york feel appropriate program poor experience greenwich program eat history emotional eater like sweets like big portion carbohydrates like chicken steak currently weigh pound ideal body weight would pound pound overweight lose excess body weight would pound weigh review systems negative head neck heart lungs gi gu orthopedic skin specifically deny chest pain heart attack coronary artery disease congestive heart failure arrhythmia atrial fibrillation pacemaker high cholesterol pulmonary embolism high blood pressure cva venous insufficiency thrombophlebitis asthma shortness breath copd emphysema sleep apnea diabetes leg foot swell osteoarthritis rheumatoid arthritis hiatal hernia peptic ulcer disease gallstones infect gallbladder pancreatitis fatty liver hepatitis hemorrhoids rectal bleed polyps incontinence stool urinary stress incontinence cancer deny cellulitis pseudotumor cerebri meningitis encephalitisphysical examination alert orient x cranial nerve iixii intact afebrile vital sign stable,"bariatrics, laparoscopic gastric bypass, weight loss programs, gastric bypass, atkin's diet, weight watcher's, body weight, laparoscopic gastric, weight loss, pounds, months, weight, laparoscopic, band, loss, diets, overweight, lost","[bariatrics, laparoscopic gastric bypass, weight loss programs, gastric bypass, atkin's diet, weight watcher's, body weight, laparoscopic gastric, weight loss, pounds, months, weight, laparoscopic, band, loss, diets, overweight, lost]","([955, 966], [458, 464], [960, 966], [985, 995])","[stroke grand